# On dynamic range: the basics
## Intro to (single) floating point precision

### Background: 
See https://en.wikibooks.org/wiki/Floating_Point/Floating-Point_Numbers for an introduction to floating point precision, range, significands and exponents.
In particular, https://en.wikibooks.org/wiki/Floating_Point/Floating_Point_Formats is a useful reference page on the precision of single vs. double floats (IEEE754 standard).

### A few warm-up examples
Just to check that we're understanding both the theory and the Python implementation correctly...

In [ ]:
import numpy as np

A single-precision float has a 24-bit significand. This gives it a relative precision of 1 in $2^{24}$, or about 1 in 16 million:

In [ ]:
print("1 in {0} ({0:g})".format(2**(24)))
print("{:.24f}".format(1./2**(24)))

Note that double-floats have a relative precision of 1 in $2^{53}$, i.e.

In [ ]:
print("1 in {0} ({0:g})".format(2**(53)))

This has some basic effects on the accuracy of addition (see also [Kahan summation](https://en.wikipedia.org/wiki/Kahan_summation_algorithm)):

In [ ]:
2**24, 2**24+1, 2**24+0.5 #Double precision, gives exact results for these examples

In [ ]:
np.float32(2**24) + np.float32(1) #Single precision, gets rounded down

In [ ]:
np.float32(2**24) + np.float32(2) #Beyond 2**24 the smallest rounding unit is 2

Of course, this scales up and down as the exponent varies:

In [ ]:
2**21 + 1./2**3 # double precision,  gives exact results for these examples

In [ ]:
np.float32(2**21) + np.float32(1./2**3)

In [ ]:
1./2**21

In [ ]:
2**2

In [ ]:
print("{:.24g}".format(np.float32(2**2) + np.float32(1./2**21)))

### Beware of aggregates
One obvious example for transient detection is averaging of visibilities. Assume for now that earlier parts of the data-acquisition / reduction process are sufficiently high-precision that we can treat them as real-number operations, but we drop down to single-float precision for visibility storage. 

We'll simply treat the combined signal as a real number, with the assumption that the important bits will generalise to complex components.

Suppose we take $2^4=32$ integrations and then attempt to average them. Now add a transient signal to one of those integrations. What is the flux of a transient that's on the borderline of being lost due to numerical precision considerations? For single-precision with a ratio of $2^{24}$, we get a borderline flux of $2^{24-4}=2^{20}$, for a dynamic range of around a million:

In [ ]:
2**20

So for a steady source flux of 1Jy, the borderline transient source fluence (in the careless / naive case) is now $\approx$1 $\mu$Jy:

In [ ]:
1./2**20

In [ ]:
steady_src_flux = 1.
n_integrations = 2**4
steady_data = steady_src_flux*np.ones(n_integrations, dtype=np.float64)
transient_signal = np.zeros_like(steady_data)
transient_fluence = 1./2**20
transient_signal[0] = transient_fluence
print(transient_signal, transient_signal.dtype) 

In [ ]:
data = steady_data + transient_signal
data

Let's recover the transient flux:

In [ ]:
np.mean(data) - steady_src_flux

And now working with single-precision:

In [ ]:
single_prec_data = np.asarray(data, dtype=np.float32)
single_prec_data 

In [ ]:
np.mean(single_prec_data)# Lost the transient to numerical precision

In [ ]:
np.mean(single_prec_data) - steady_src_flux

So, if we were searching for the faint transient in the time-averaged (single-precision) image, we'd have no hope - the averaging process has diluted the flux to below the $\frac{1}{2^{24}}$ precision-ratio limit. The takeaway being that we need to think in terms of **fluence ratios** (or equivalently, time-averaged flux-ratios) for the search-averaging period.

If we search for transient on the timescale of a single integration, we'll still detect it, so simply need to beware of aggregate statistics:

In [ ]:
residuals = single_prec_data - np.mean(single_prec_data)
residuals

Note, we could alternatively use higher-precision for aggregate computation:

In [ ]:
np.mean(single_prec_data, dtype=np.float64)

In [ ]:
# For comparison:
transient_fluence / n_integrations

## Summary:
For single-precision floating point, the 'instantaneous' dynamic range is 1 in 16 million. This is preserved so long as care is taken when dealing with aggregate quantities...

... Another example of which is the fourier transform (see next notebook).